In [1]:
#一共有1326个句子，32个intent，
import itertools
import numpy as np

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import logging
import numpy
import pickle
import torch
from torch.optim import Adam
import json
import matplotlib as plt
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import statsmodels.api as sm 
import random 
import jieba
import Ipynb_importer
import Untitled1
from Untitled1 import preprocess_data, generate_id, load_pretrained_word2vec
from Untitled1 import generate_intent2text_intent_texts_len, generate_support_query
from Untitled1 import sample_sq, dict_2_2array, Fewshot



importing Jupyter notebook from Untitled1.ipynb


In [3]:
def _init_logging_handler(self):
        current_time = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())

        stderr_handler = logging.StreamHandler()
        file_handler = logging.FileHandler('./log/log_{}.txt'.format(current_time))
        logging.basicConfig(handlers=[stderr_handler, file_handler])
        logger = logging.getLogger()
        logger.setLevel(logging.INFO)
_init_logging_handler()    

TypeError: _init_logging_handler() missing 1 required positional argument: 'self'

In [ ]:
with open('gongan_train_10.17.json') as f:
    load_json = json.load(f)
    data = load_json['rasa_nlu_data']['common_examples']
print(len(data))

In [ ]:
data1,intentlist = preprocess_data(data)#把同义句低于5个的intent删除
generate_id(data1)#生成word2id,id2word,intent2id,id2intent，jieba分词

In [ ]:
random.shuffle(data1)

In [ ]:
#转化成数字label
with open('intent2id','rb') as f:
    intent2id = pickle.load(f)
with open('word2id','rb') as f:
    word2id = pickle.load(f)   
for item in data1:
    item['intent'] = intent2id[item['intent']]
    item['text'] = [word2id[w] for w in item['text']]

In [ ]:
import random
id_a = list(range(len(intentlist)))
id_t = random.sample(id_a, int(0.75 * len(intentlist)))
id_v = list(set(id_a)-set(id_t))
intent_train = [intent2id[intentlist[i]] for i in id_t]
intent_test = [intent2id[intentlist[i]] for i in id_v]

In [ ]:
print(intent_train)

In [ ]:
#做一个intent的同义句集合(intent_same记录同义句数量)，集合i位置代表intent_i的同义句有多少个，。筛选intent同义句大于15个的data条数，
intent_same = [0 for i in range(355)]
#print(intent_same)
#train_data1筛选出intent同义句>15
train_data = []
test_data = []
for item in data1:
    if item['intent'] in intent_train:
        train_data.append(item)
    else:
        test_data.append(item)
print(len(train_data),len(test_data))


In [ ]:
#检查text的长度分布,截断长度
def len_cut_20(data):
    text_len = [0 for i in range(100)]
    for item in data:
        if(len(item['text']))<100:
            text_len[len(item['text'])]+=1
    max_len = 20
    for item in data:
        length = len(item['text'])
        if length<20:
            item['text'].extend([0] * (max_len - length))
        else:
            item['text'] = item['text'][:20]
len_cut_20(train_data)
len_cut_20(test_data)

In [ ]:
#构建最终数据,生成传入模型的intent2text，intent_texts_len
intent2text, intent_texts_len = generate_intent2text_intent_texts_len(train_data)
t_intent2text,t_intent_texts_len = generate_intent2text_intent_texts_len(test_data)

In [ ]:
eposide_iter = int((28*27*26*25*24)/(5*4*3*2))
way = 5
shot = 10
query = 20
dynamic_iter = 3
k = 100
d_a = 64
emb = 300
hid=128
lr =  0.0001
PATH = './model_iter2'

In [ ]:
with open('renmin_emb_weights','rb') as f:
    emb_weights = pickle.load(f)
with open('word2id','rb') as f:
    word2id = pickle.load(f)  #2889个词

In [ ]:
train_sample_classes = list(itertools.combinations(intent_train,5))
print(len(train_sample_classes))
print(eposide_iter)

In [ ]:
model = Fewshot(num_words=len(word2id),embed_size=emb,hidden_size=hid,emb_weight =emb_weights,
                   way=way,shot=shot,query=query,dynamic_iter=dynamic_iter,k=k,d_a=d_a)


model.load_state_dict(torch.load('./model/42500.pt'))
model.cuda()
optim = Adam(lr=lr, params=filter(lambda x: x.requires_grad, model.parameters()),weight_decay=5e-4)
train_sample_classes = list(itertools.combinations(intent_train,5))
test_sample_classes = list(itertools.combinations(intent_test,5))


for training_iter in range(1):
    for i in range(len(train_sample_classes)):       
        sample_classes = train_sample_classes[i]
        if  training_iter>0 and i==0:
            lr = lr*0.5
            optim = Adam(lr=lr, params=filter(lambda x: x.requires_grad, model.parameters()),weight_decay=5e-4)
        
        optim.zero_grad()
        support_ways_text,support_ways_text_lens,query_ways_text,query_ways_text_lens = \
            generate_support_query(intent2text, intent_texts_len, sample_classes, way, shot, query)

        loss = model(support_ways_text.t(),support_ways_text_lens,query_ways_text.t(),query_ways_text_lens,i)

        if i%500==0:        
            logging.debug('---------------')
            logging.debug('iter %d loss %d'%(i,loss))
        loss.backward()
        optim.step()
        
        if i%500==0:
            loss = 0
            for j in range(len(test_sample_classes)):
                support_ways_text,support_ways_text_lens,query_ways_text,query_ways_text_lens = \
                    generate_support_query(t_intent2text, t_intent_texts_len, test_sample_classes[j], way, shot, query)
                loss += model(support_ways_text.t(),support_ways_text_lens,query_ways_text.t(),query_ways_text_lens,j)
            loss/=len(test_sample_classes)
            logging.debug('iter %d val_loss %d'%(i,loss))
            torch.save(model.state_dict(), PATH + '/' + str(i) + '.pt')


In [ ]:
#测试

model = Fewshot(num_words=len(word2id),embed_size=emb,hidden_size=hid,emb_weight =emb_weights,
                   way=way,shot=shot,query=query,dynamic_iter=dynamic_iter,k=k,d_a=d_a)
PATH='./model_new/10000.pt'
model.load_state_dict(torch.load(PATH))
model.cuda()

n_Classes_vectors = []
for i in range(5): #试情况而定，取均值
    support_idx, query_idx = {},{}
    support_ways_text,support_ways_text_lens = [],[]
    query_ways_text,query_ways_text_lens = [],[]

    support_idx = all_sq(intent2text,intent_texts_len,way,shot) 

    with open('id2intent','rb') as f:
        id2intent = pickle.load(f)  #int2str
    with open('id2word','rb') as f:
        id2word = pickle.load(f)

    support_ways_text,support_ways_text_lens = dict_2_2array(support_idx)
    support_ways_text = support_ways_text.cuda()#[140*shot,text_length]
    support_ways_text_lens = support_ways_text_lens#.cuda()
    n_Classes_vectors.append(model.get_classes_vectors(support_ways_text.t(),support_ways_text_lens).squeeze())
    
    #[classes, 256]
Classes_vector = n_Classes_vectors[0]+n_Classes_vectors[1]+n_Classes_vectors[2]+n_Classes_vectors[3]+n_Classes_vectors[4]
print(Classes_vector.size())
with open('Classes_vectors','wb') as f:
    pickle.dump(Classes_vector,f)


In [ ]:
import pandas as pd
import sys
import os
df = pd.read_excel('./gongan_test.xlsx')
df.head()
columns = df.columns
test_text_list, test_label_list = [],[]
values = df[columns[0]].values
for idx, item in enumerate(values):
    test_text_list.append(item)
values = df[columns[1]].values
for idx, item in enumerate(values):
    test_label_list.append(item)

In [ ]:
#test model
from Untitled1 import Fewshot
import jieba
max_len = 20
#query_text = '请问客服在吗,我找他？','好的'
query_text = tuple(test_text_list)
with open('word2id','rb') as f:
    word2id = pickle.load(f)
query = []
query_lens = []

for text_i in query_text:
    text_i = jieba.cut(text_i,cut_all=False)
    text_i = (list(text_i))
    text_i = [word2id[w] if (w in word2id) else (1) for w in text_i]
    l = len(text_i) 
    if l<20:
        text_i.extend([0] * (max_len - l))
    else: 
        text_i = text_i[:20]
    query.append(text_i)#[q_nums,max_len]
    if l<20: after_l = l
    else: after_l=20
    query_lens.append(after_l)
#query构建完毕
query = torch.tensor(query).cuda()
query_lens = torch.tensor(query_lens)#.cuda()

q_scores = model.get_classify_result(Classes_vector, query.t(),query_lens)
print(q_scores.size())
result_index = torch.max(q_scores, 0)[1]
q_scores = q_scores.t()

#对应classes：

with open('id2intent','rb') as f:
    id2intent = pickle.load(f)

intent_class_vector = [id2intent[int(i)] for i in intent2text.keys()]

   
q_classes = [id2intent[i.item()] for i in result_index]

num_r = 0
for i in range(len(test_text_list)):  
    if q_classes[i]!=test_label_list[i]:  
        #print('---------------------------------')   
        #print(query_text[i])
        #print('预测', q_classes[i],'真值',test_label_list[i])
        above095 = [j for j in range(32) if q_scores[i][j]>0.95]
        #above095.sort(reverse=True)
        #print('所有可能答案-----')
        #for k in above095: print(intent_class_vector[k],q_scores[i][k])
        
        
        num_r+=1
print(num_r)

